In [ ]:
import torch
import os
import numpy as np
import skimage

os.chdir('/homes/dnogina/code/topology-control/')

In [ ]:
from src.CPipelineOrchestrator import CPipelineOrchestrator
from src.CModelTrainer import SDFDataset
from src.CEvaluator import CEvaluator
from src.CGeometryUtils import VolumeProcessor

In [ ]:
ckpt = torch.load('artifacts/first_working.pth')

In [ ]:
orc = CPipelineOrchestrator()
deepsdf_model = orc.architecture_manager.get_model()
_ = deepsdf_model.load_state_dict(ckpt['model_state_dict'])

In [ ]:
processing_results = orc.data_processor.generate_sdf_dataset(
                z_dim=16,
                latent_mean=0,
                latent_sd=0.02
            )

In [ ]:
ds = SDFDataset(processing_results['dataset_info'])
ds.latent_vectors = ckpt['latent_vectors']

In [ ]:
ds.latent_vectors.min(), ds.latent_vectors.max()

In [ ]:
import os
os.chdir('/homes/dnogina/code/topology-control/volume/')

In [ ]:
from visualize import visualize_latent_vs_genera, construct_from_latent
from config import COORDS_FIRST

In [ ]:
visualize_latent_vs_genera(path='/homes/dnogina/code/topology-control/volume/data/2d_latents_volumes_deepsdf.npz')

In [ ]:
from compute_path_opt import compute_path
from model import Latent2Genera
from dataset import GeneraDataset

dataset = GeneraDataset(
        dataset_path=os.path.join( "data", "2d_latents_volumes_deepsdf.npz")
    )

In [ ]:
checkpoint = torch.load("checkpoints/latent2genera_deefsdf_best.pt", map_location="cuda")

In [ ]:
model = Latent2Genera(16, num_classes=dataset.num_classes).to("cuda")

# Load the saved state_dict (weights)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

In [ ]:
path = compute_path(ds.latent_vectors[-2].to("cuda"), ds.latent_vectors[-1].to("cuda"), model)

In [ ]:
path.shape

In [ ]:
import igl
from visualize import get_volume_coords #,generate_mesh
from compute_volume import predict_sdf

def construct_from_latent(model, latent, positioning, index, type):
    """
    Reconstruct the mesh from a latent vector using the model
    
    Parameters:
        model: Trained DeepSDF model or SDF_interpolator object
        latent: A latent vector
        positioning: helix (circular) or row
        index: the index used for angle or shift calculation
    Returns:
        vertices, faces
    """
    coords, grid_size = get_volume_coords()

    # vertices, faces = generate_mesh(latent.to("cuda"), coords[None].to("cuda"), grid_size, model.to("cuda"), type)

    if positioning == "row":
        vertices[:, 0] += 2 * index
    else:
        angle = (index % 10) * 2 * np.pi / 10 
        vertices[:, 0] += 3 * np.cos(angle)
        vertices[:, 1] += 3 * np.sin(angle)
        vertices[:, 2] += 0.2 * index
    
    return vertices, faces

i = 49
latent = path[i]
V, F = construct_from_latent(deepsdf_model, latent, "helix", i, COORDS_FIRST)


In [ ]:
import plotly.graph_objects as go
import numpy as np
from trimesh import Trimesh

import plotly.io as pio
pio.renderers.default='notebook'

x, y, z = -np.array(V).T
i, j, k = np.array(F).T

fig = go.Figure(data=[
    go.Mesh3d(x=x, y=y, z=z, i=i, j=j, k=k, opacity=1),
    # go.Scatter3d(x=coords[:,0], y=coords[:,1], z=coords[:,2], mode='markers', marker=dict(size=3, color=sdfs),marker_colorscale='Viridis')
])
fig.update_layout(template='plotly_white')
fig.show()
